<a href="https://colab.research.google.com/github/rozhix/LP-detection/blob/master/LP_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Sep 21 07:43:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Clone Darknet
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15308, done.
remote: Total 15308 (delta 0), reused 0 (delta 0), pack-reused 15308
Receiving objects: 100% (15308/15308), 13.69 MiB | 20.17 MiB/s, done.
Resolving deltas: 100% (10399/10399), done.


In [ ]:
# Configure Darknet
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile DarkNet
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [ ]:
# Create .names files
!echo -e 'License Plate\nWithout License Plate' > data/obj.names

In [ ]:
# Create .data file 
!echo -e 'classes=2\ntrain = data/train.txt\nvalid =data/test.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/' > data/obj.data

In [ ]:
# Create a folder and unzip image dataset
#!mkdir data/obj
!unzip /content/drive/MyDrive/images.zip -d data/obj

Archive:  /content/drive/MyDrive/images.zip
   creating: data/obj/images/
   creating: data/obj/images/test/
  inflating: data/obj/images/test/Cars125_png.rf.70e18062434e3bf60b556c7ba7819493.jpg  
  inflating: data/obj/images/test/Cars125_png.rf.70e18062434e3bf60b556c7ba7819493.txt  
  inflating: data/obj/images/test/Cars134_png.rf.d03b7a9a64651d4ca0f8063284a75400.jpg  
  inflating: data/obj/images/test/Cars134_png.rf.d03b7a9a64651d4ca0f8063284a75400.txt  
  inflating: data/obj/images/test/Cars13_png.rf.234bf12615486ac19752e4b61a04a7e2.jpg  
  inflating: data/obj/images/test/Cars13_png.rf.234bf12615486ac19752e4b61a04a7e2.txt  
  inflating: data/obj/images/test/Cars140_png.rf.167a72cf610f469d99778535e27be40c.jpg  
  inflating: data/obj/images/test/Cars140_png.rf.167a72cf610f469d99778535e27be40c.txt  
  inflating: data/obj/images/test/Cars142_png.rf.812b806357d5831c4713809344013503.jpg  
  inflating: data/obj/images/test/Cars142_png.rf.812b806357d5831c4713809344013503.txt  
  inflating: 

In [ ]:
# Create train.txt file
import glob
images_list = glob.glob('data/obj/images/train/*.jpg')
with open("data/train.txt", 'w') as f:
  f.write('\n'.join(images_list))

In [ ]:
# Download pre-trained weights for the convolutional layers file
!wget https://pjreddie.com/media/files/darknet53.conv.74 

--2021-09-21 07:41:31--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  53.9MB/s    in 2.9s    

2021-09-21 07:41:35 (53.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/yolov3_training_last.weights -dont_show  

Streaming output truncated to the last 5000 lines.
 total_bbox = 35216, rewritten_bbox = 0.000000 % 

 5926: 0.129423, 0.091228 avg loss, 0.001000 rate, 21.237322 seconds, 379264 images, 0.660539 hours left
Loaded: 0.000106 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.938494), count: 1, class_loss = 0.000030, iou_loss = 0.001582, total_loss = 0.001612 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000006, iou_loss = 0.000000, total_loss = 0.000006 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.771530), count: 2, class_loss = 0.002303, iou_loss = 0.048004, total_loss = 0.050308 
 total_bbox = 35219, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.697546), count: 1, class_loss = 0.000000, iou_loss = 0.050748, total_loss = 0.050749 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.